# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff406575790>
├── 'a' --> tensor([[-1.1621, -1.4581, -0.3053],
│                   [ 1.8814, -0.1467, -0.5022]])
└── 'x' --> <FastTreeValue 0x7ff406575670>
    └── 'c' --> tensor([[-3.1926e-01,  2.3200e-04, -1.4527e+00, -4.8066e-01],
                        [-2.1911e-01,  2.1841e+00,  1.0091e+00,  9.8179e-01],
                        [ 5.0137e-01, -1.2766e-01,  3.0344e+00,  1.0387e+00]])

In [4]:
t.a

tensor([[-1.1621, -1.4581, -0.3053],
        [ 1.8814, -0.1467, -0.5022]])

In [5]:
%timeit t.a

76.3 ns ± 0.461 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff406575790>
├── 'a' --> tensor([[ 1.4254, -1.6790,  1.9003],
│                   [-0.6092, -0.1756, -0.9940]])
└── 'x' --> <FastTreeValue 0x7ff406575670>
    └── 'c' --> tensor([[-3.1926e-01,  2.3200e-04, -1.4527e+00, -4.8066e-01],
                        [-2.1911e-01,  2.1841e+00,  1.0091e+00,  9.8179e-01],
                        [ 5.0137e-01, -1.2766e-01,  3.0344e+00,  1.0387e+00]])

In [7]:
%timeit t.a = new_value

91 ns ± 0.413 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.1621, -1.4581, -0.3053],
               [ 1.8814, -0.1467, -0.5022]]),
    x: Batch(
           c: tensor([[-3.1926e-01,  2.3200e-04, -1.4527e+00, -4.8066e-01],
                      [-2.1911e-01,  2.1841e+00,  1.0091e+00,  9.8179e-01],
                      [ 5.0137e-01, -1.2766e-01,  3.0344e+00,  1.0387e+00]]),
       ),
)

In [10]:
b.a

tensor([[-1.1621, -1.4581, -0.3053],
        [ 1.8814, -0.1467, -0.5022]])

In [11]:
%timeit b.a

70 ns ± 0.393 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.4391, -1.1123,  1.0589],
               [ 1.1947,  0.2127,  0.1321]]),
    x: Batch(
           c: tensor([[-3.1926e-01,  2.3200e-04, -1.4527e+00, -4.8066e-01],
                      [-2.1911e-01,  2.1841e+00,  1.0091e+00,  9.8179e-01],
                      [ 5.0137e-01, -1.2766e-01,  3.0344e+00,  1.0387e+00]]),
       ),
)

In [13]:
%timeit b.a = new_value

584 ns ± 2.93 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

965 ns ± 21 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.9 µs ± 47.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

167 µs ± 505 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

165 µs ± 768 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff40656fa60>
├── 'a' --> tensor([[[-1.1621, -1.4581, -0.3053],
│                    [ 1.8814, -0.1467, -0.5022]],
│           
│                   [[-1.1621, -1.4581, -0.3053],
│                    [ 1.8814, -0.1467, -0.5022]],
│           
│                   [[-1.1621, -1.4581, -0.3053],
│                    [ 1.8814, -0.1467, -0.5022]],
│           
│                   [[-1.1621, -1.4581, -0.3053],
│                    [ 1.8814, -0.1467, -0.5022]],
│           
│                   [[-1.1621, -1.4581, -0.3053],
│                    [ 1.8814, -0.1467, -0.5022]],
│           
│                   [[-1.1621, -1.4581, -0.3053],
│                    [ 1.8814, -0.1467, -0.5022]],
│           
│                   [[-1.1621, -1.4581, -0.3053],
│                    [ 1.8814, -0.1467, -0.5022]],
│           
│                   [[-1.1621, -1.4581, -0.3053],
│                    [ 1.8814, -0.1467, -0.5022]]])
└── 'x' --> <FastTreeValue 0x7ff357206340>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

38.1 µs ± 233 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff35718ee50>
├── 'a' --> tensor([[-1.1621, -1.4581, -0.3053],
│                   [ 1.8814, -0.1467, -0.5022],
│                   [-1.1621, -1.4581, -0.3053],
│                   [ 1.8814, -0.1467, -0.5022],
│                   [-1.1621, -1.4581, -0.3053],
│                   [ 1.8814, -0.1467, -0.5022],
│                   [-1.1621, -1.4581, -0.3053],
│                   [ 1.8814, -0.1467, -0.5022],
│                   [-1.1621, -1.4581, -0.3053],
│                   [ 1.8814, -0.1467, -0.5022],
│                   [-1.1621, -1.4581, -0.3053],
│                   [ 1.8814, -0.1467, -0.5022],
│                   [-1.1621, -1.4581, -0.3053],
│                   [ 1.8814, -0.1467, -0.5022],
│                   [-1.1621, -1.4581, -0.3053],
│                   [ 1.8814, -0.1467, -0.5022]])
└── 'x' --> <FastTreeValue 0x7ff35718e340>
    └── 'c' --> tensor([[-3.1926e-01,  2.3200e-04, -1.4527e+00, -4.8066e-01],
                        [-2.1911e-01,  2.1841e+00,  1.0091e+00,

In [23]:
%timeit t_cat(trees)

38.9 µs ± 1.53 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

62.2 µs ± 270 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.1621, -1.4581, -0.3053],
                [ 1.8814, -0.1467, -0.5022]],
       
               [[-1.1621, -1.4581, -0.3053],
                [ 1.8814, -0.1467, -0.5022]],
       
               [[-1.1621, -1.4581, -0.3053],
                [ 1.8814, -0.1467, -0.5022]],
       
               [[-1.1621, -1.4581, -0.3053],
                [ 1.8814, -0.1467, -0.5022]],
       
               [[-1.1621, -1.4581, -0.3053],
                [ 1.8814, -0.1467, -0.5022]],
       
               [[-1.1621, -1.4581, -0.3053],
                [ 1.8814, -0.1467, -0.5022]],
       
               [[-1.1621, -1.4581, -0.3053],
                [ 1.8814, -0.1467, -0.5022]],
       
               [[-1.1621, -1.4581, -0.3053],
                [ 1.8814, -0.1467, -0.5022]]]),
    x: Batch(
           c: tensor([[[-3.1926e-01,  2.3200e-04, -1.4527e+00, -4.8066e-01],
                       [-2.1911e-01,  2.1841e+00,  1.0091e+00,  9.8179e-01],
                       [ 5.0137e-01, -1

In [26]:
%timeit Batch.stack(batches)

89.4 µs ± 271 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.1621, -1.4581, -0.3053],
               [ 1.8814, -0.1467, -0.5022],
               [-1.1621, -1.4581, -0.3053],
               [ 1.8814, -0.1467, -0.5022],
               [-1.1621, -1.4581, -0.3053],
               [ 1.8814, -0.1467, -0.5022],
               [-1.1621, -1.4581, -0.3053],
               [ 1.8814, -0.1467, -0.5022],
               [-1.1621, -1.4581, -0.3053],
               [ 1.8814, -0.1467, -0.5022],
               [-1.1621, -1.4581, -0.3053],
               [ 1.8814, -0.1467, -0.5022],
               [-1.1621, -1.4581, -0.3053],
               [ 1.8814, -0.1467, -0.5022],
               [-1.1621, -1.4581, -0.3053],
               [ 1.8814, -0.1467, -0.5022]]),
    x: Batch(
           c: tensor([[-3.1926e-01,  2.3200e-04, -1.4527e+00, -4.8066e-01],
                      [-2.1911e-01,  2.1841e+00,  1.0091e+00,  9.8179e-01],
                      [ 5.0137e-01, -1.2766e-01,  3.0344e+00,  1.0387e+00],
                      [-3.1926e-01,  2.3200e-

In [28]:
%timeit Batch.cat(batches)

164 µs ± 2.6 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

386 µs ± 9.85 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
